<a href="https://colab.research.google.com/github/j1sk1ss/j1sk1ss/blob/main/ITcup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
import gensim.downloader as api
import numpy as np
import torch
import string
import transformers
import gensim
from transformers import DistilBertModel, DistilBertTokenizer, logging

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

import matplotlib.pyplot as plt

nltk.download('stopwords')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
with open("drive/MyDrive/ITcup/ranking_train.jsonl", "r") as f:
    data = [json.loads(line) for line in f]

In [ ]:
data=data[:1000]

In [ ]:
stop_words = stopwords.words('english')
for i in range(len(stop_words)):
  stop_words[i] = "".join([char for char in stop_words[i] if char not in "'"])

porter = PorterStemmer()

In [ ]:
nltk.download('punkt')
logging.set_verbosity_error()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
scores   = []
for i in range(len(data)):
  for j in range(len(data[i]['comments'])):
    scores.append(data[i]['comments'][j]['score'])

exclamations = []
questions = []
links = []
lengths = []
for i in range(len(data)):
  avg_com_len=0
  for j in range(len(data[i]['comments'])):
    avg_com_len += len(data[i]['comments'][j]['text'])
  avg_com_len/=5
  tmp_link = 0
  tmp_length = 0
  for j in range(len(data[i]['comments'])):
    tmp_length = len(data[i]['comments'][j]['text'])/avg_com_len
    tmp_exclamation = data[i]['comments'][j]['text'].count("!")#/tmp_length
    tmp_question = data[i]['comments'][j]['text'].count("?")#/tmp_length
    if ("http" or "www") in data[i]['comments'][j]['text']:
      tmp_link += 1

  exclamations.append(tmp_exclamation)
  questions.append(tmp_question)
  links.append(tmp_link)
  lengths.append(tmp_length)

In [ ]:
import pandas as pd
df = pd.DataFrame(list(zip(scores[:1000], exclamations, questions, links, lengths)), columns =['Score', '!', '?', 'Has Links', 'Length'])

In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

,Score,!,?,Has Links,Length
Score,1.000000,-0.042213,0.035056,-0.031487,-0.009955
!,-0.042213,1.000000,0.003894,-0.025979,0.070310
?,0.035056,0.003894,1.000000,0.005149,0.149402
Has Links,-0.031487,-0.025979,0.005149,1.000000,-0.002232
Length,-0.009955,0.070310,0.149402,-0.002232,1.000000


In [ ]:
for datum in data:
  datum['text'] = datum['text'].lower()
  datum['text'] = "".join([char for char in datum['text'] if char not in string.punctuation])
  datum['text'] = word_tokenize(datum['text'])
  datum['text'] = [word for word in datum['text'] if word not in stop_words]
  #datum['text'] = [porter.stem(word) for word in datum['text']]
  #datum['text'] = ' '.join(datum['text'])
  for comment in datum['comments']:
    comment['text'] = comment['text'].lower()
    comment['text'] = "".join([char for char in comment['text'] if char not in string.punctuation])
    comment['text'] = word_tokenize(comment['text'])
    comment['text'] = [word for word in comment['text'] if word not in stop_words]
    #comment['text'] = [porter.stem(word) for word in comment['text']]
    #comment['text'] = ' '.join(comment['text'])

In [ ]:
texts = []
comments = []

for i in range(len(data)):
  texts.append(data[i]['text'])
  for j in range(len(data[i]['comments'])):
    comments.append(data[i]['comments'][j]['text'])

In [ ]:
from gensim.models import Word2Vec
import gensim.downloader as api
model = api.load("glove-wiki-gigaword-100")

In [ ]:
similarity=[]

sentences=comments[0:5]
target_sentence=texts[0]

for i in range(len(texts)):
  sentences_similarity = np.zeros(len(sentences))
  w2v_vocab = model.vocab.keys()
  sentences=comments[5*i:5+5*i]
  target_sentence=texts[i]
  for idx, sentence in enumerate(sentences):
    sentence_words = [w for w in sentence if w in w2v_vocab]
    if (len(sentence_words) == 0):
      sentence_words=['fortune']
    target_sentence_words = [w for w in target_sentence if w in w2v_vocab]
    if (len(target_sentence_words) == 0):
      target_sentence_words=['fortune']
    sim = model.n_similarity(target_sentence_words, sentence_words)
    sentences_similarity[idx] = sim

  result = list(zip(sentences_similarity))
  #result.sort(key=lambda item:item[0], reverse=True)
  for i in range(5):
    similarity+=result[i]
  #print("Target:", target_sentence)
  #print(result[0][0])

In [ ]:
#similarity_normalized = similarity.copy()
#for i in range(int(len(similarity))):
#  for j in range(5):
#    similarity_normalized[5*i+j] = similarity[5*i+j] / max(similarity[5*i:5+5*i])
#df['Similarity_normalized'] = similarity_normalized

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(128, 64),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.1),
            nn.Linear(64,32),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.1),
            nn.Linear(32,16),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.1),
            nn.Linear(16,5)
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
model = api.load("glove-wiki-gigaword-100")
w2v_vocab = model.vocab.keys()

In [ ]:
comms=[]
for i in range(len(comments)):
  if (comments[i] and comments[i][0] in w2v_vocab):
    cur = model.wv.get_vector(comments[i][0])
    for j in range(len(comments[i])):
      if (comments[i][j] and comments[i][j] in w2v_vocab):
        cur = np.sum([cur, model.wv.get_vector(comments[i][j])], axis = 0)
  comms.append(cur/(len(comments[i])))

<ipython-input-288-c29fed2aa1ce>:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  cur = model.wv.get_vector(comments[i][0])
<ipython-input-288-c29fed2aa1ce>:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  cur = np.sum([cur, model.wv.get_vector(comments[i][j])], axis = 0)
<ipython-input-288-c29fed2aa1ce>:8: RuntimeWarning: divide by zero encountered in true_divide
  comms.append(cur/(len(comments[i])))


In [ ]:
posts=[]
for i in range(len(texts)):
  if (texts[i] and texts[i][0] in w2v_vocab):
    cur = model.wv.get_vector(texts[i][0])
    for j in range(len(texts[i])):
      if (texts[i][j] and texts[i][j] in w2v_vocab):
        cur = np.sum([cur, model.wv.get_vector(texts[i][j])], axis = 0)
  posts.append(cur/(len(texts[i])))

<ipython-input-289-a55d8622a957>:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  cur = model.wv.get_vector(texts[i][0])
<ipython-input-289-a55d8622a957>:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  cur = np.sum([cur, model.wv.get_vector(texts[i][j])], axis = 0)


In [ ]:
expected = [
([1,0,0,0,0],
[0,1,0,0,0],
[0,0,1,0,0],
[0,0,0,1,0],
[0,0,0,0,1],)*(int(len(comms)/5)+5)
]
expected[0] = expected[0][:len(comms)]

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
!pip install tqdm
from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
loss_fn = nn.CrossEntropyLoss()

model = NeuralNetwork()
optimizer = optim.SGD(model.parameters(), lr=0.01)

for epoch in range(5):
  for i in tqdm(range(len(expected[0]))):
    outputs = model.forward(torch.FloatTensor(np.concatenate((comms[i], np.array([similarity[i]] * 28)))))
    expected_flat = torch.FloatTensor(expected[0][i]).flatten()
    loss = loss_fn(outputs, expected_flat)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    pass

100%|██████████| 5000/5000 [00:04<00:00, 1132.51it/s]


In [ ]:
for i in range(len(comms)):
  outputs = model.forward(torch.FloatTensor(comms[i]))
  expected_flat = torch.FloatTensor(expected[0][i]).flatten()


RuntimeError: ignored